Скрипт для соединения двух датасетов, скачанных с auto.ru в разное время.
Если вы планируете использовать скрипт в своих целях, как инструмент для конкатенации нескольких датасетов, полученных после запуска парсера  auto_ru_parser, то вам стоит пропустить этап "Удаление не корректных данных по ПТС".

In [49]:
import pandas as pd
import numpy as np
from datetime import datetime

DIR_DATA = 'parsing_data/'
DIR_TEST = 'data/'

In [8]:
df_1 = pd.read_csv(DIR_DATA + 'cars_prices_multiprocessing.csv')
df_2 = pd.read_csv(DIR_DATA + 'cars_prices_multiprocessing_08_11_2020.csv')

print(f"df_1: {df_1.shape}")
print(f"df_2: {df_2.shape}")

df_1: (162913, 35)
df_2: (56457, 36)


In [10]:
# В данные попадает аж 187 раз заголовки для csv-таблицы. Удалим их
df_1 = df_1.loc[df_1.brand != 'brand']
print(f"New size df_1: {df_1.shape}")

df_2 = df_2.loc[df_2.brand != 'brand']
print(f"New size df_1: {df_2.shape}")

New size df_1: (161714, 35)
New size df_1: (55508, 36)


In [24]:
# Проверим наличие дубликатов по sell_id
print(f"Shape df_1 without duplicates: {df_1.sell_id.unique().shape}")
print(f"Shape df_2 without duplicates: {df_2.sell_id.unique().shape}")

Shape df_1 without duplicates: (55541,)
Shape df_2 without duplicates: (54043,)


В первом датасете действительно содержится много дубликатов. Это результат моей не внимательности.  
Запускал скрипт пасринга несколько раз без очистки csv-файла, в который выполнялась записи данных.

In [12]:
# Удалим дубли по признаку sell_id
df_1.drop_duplicates(subset=['sell_id'], keep='last', inplace=True)
print(f"New size: {df_1.shape}")

df_2.drop_duplicates(subset=['sell_id'], keep='last', inplace=True)
print(f"New size: {df_2.shape}")

New size: (55884, 35)
New size: (54256, 36)


### Первичная обработка данных

При парсинге первого датасета df_1 мной была допущена ошибка при получении значения ПТC.  
Она привела к тому, что значения Дубликат нет совсем, вместо него записано значение Оригинал.  
Но мне повезло, что есть второй датасет, собранный немного позже. Второй датасет в большей части дублируем информацию из первого.  
Так же в тестовом датасете могут содержаться теже записи, которые есть в первом датасета.  

Конечно не хочется терять данные, но значение признака price, сильно меняется от значения признака ПТС.  
Поэтому лучше потерять часть данных, но восстановить информативность сотавшихся.


In [15]:
df_2.isnull().sum()

bodyType                    2
brand                       0
color_hex                   0
description              1639
engineDisplacement          2
enginePower                 2
equipment_dict          12339
fuelType                    2
image                       0
mileage                     0
modelDate                   2
model_info                  0
model_name                  0
name                        2
numberOfDoors               2
parsing_unixtime            0
priceCurrency             211
productionDate              0
sell_id                     0
section                     0
url_saleid                  0
super_gen                   2
vehicleConfiguration        2
vehicleTransmission         2
vendor                      0
Владельцы               12799
ПТС                       333
Привод                      2
Руль                        0
Состояние                 886
Таможня                     0
price                     211
price_timestamp           211
auto_class

In [16]:
# удалим объекты с пропусками в след. признаках, а так же где нет цены, так как кол-во пропусков там минимально
cols_to_dropna = 'bodyType engineDisplacement enginePower fuelType modelDate name numberOfDoors super_gen vehicleConfiguration vehicleTransmission Привод price price_timestamp price_segment'.split(' ')


df_1 = df_1.dropna(subset=cols_to_dropna)
df_2 = df_2.dropna(subset=cols_to_dropna)

In [17]:
# переведем признаки price и sell_id в формат int64
df_1.price = df_1.price.astype('int64')
df_1.sell_id = df_1.sell_id.astype('int64')

df_2.price = df_2.price.astype('int64')
df_2.sell_id = df_2.sell_id.astype('int64')

In [21]:
# Посмотрим влияет ли признак ПТС на таргет
pd.pivot_table(df_2, values=['price'], index=['ПТС'],
               aggfunc=[np.mean, np.median]).astype('int64')

,mean,median
,price,price
ПТС,,
DUPLICATE,606521,420000
ORIGINAL,2091804,1050000


Как видно из сводной таблицы, по среднему разница более чем в 3 раза, по медиане разница более чем в 2.5 раза.  
Это подтверждает мое решение пожертвовать частью данных, где этот признак получен с не верным значением.

### Удаление не корректных данных по  ПТС

Прочитаем тестовые данные

In [26]:
test = pd.read_csv(DIR_TEST+'test_26_10_2020.csv')
test.shape

(34686, 32)

In [28]:
###
# Посмотрим какое кол-во объявлений из трейна есть в тесте.  Это нужно для того, чтобы постараться по максимуму свосстановить корректные значения признака ПТС в трейне из теста по признаку sel_id

train_1_ids = df_1.sell_id.unique()
train_2_ids = df_2.sell_id.unique()
test_ids = test.sell_id.unique()

diff_train_1_train_2_ids = set(train_1_ids) - set(train_2_ids)
common_tr_1_tr_2_and_test_ids = set(diff_train_1_train_2_ids) & set(test_ids)

print(f"В тесте есть: {len(common_tr_1_tr_2_and_test_ids)} записей содержащиъся в df_1")

# Сокращаем df_1 до тех записей, которые есть в common_tr_1_tr_2_and_test_ids
df_1 = df_1[df_1.sell_id.isin(common_tr_1_tr_2_and_test_ids)]
df_1.shape

В тесте есть: 3661 записей содержащиъся в df_1


In [29]:
# Теперь пройдемся по каждой строке df_1 и заменим значение признака ПТС из теста
def fill_pts_value_from_test(row):
    curr_sell_id = row['sell_id']
    
    if curr_sell_id in test_ids:
        pts_in_test = test[test.sell_id == curr_sell_id].ПТС.iloc[0]
        row['ПТС'] = pts_in_test
    
    return row

df_1 = df_1.apply(fill_pts_value_from_test, axis=1)

In [30]:
# проверим результат
df_1.ПТС.value_counts()

Оригинал    3240
Дубликат     421
Name: ПТС, dtype: int64

### Объединение df_1 и df_2
В процессе отладки парсера, были приняты решения от некоторых признаков отказаться и добавить другие.  
Ввиду этого размерность df_1 и df_2 не совпадают.  
Удалим не нужные признаки

In [31]:
# для df_1
cols_to_del_df_1 = ['model', 'price_timestamp']
df_1.drop(labels=cols_to_del_df_1, axis=1, inplace=True)

# для df_2
cols_to_del_df_2 = ['price_timestamp']
df_2.drop(labels=cols_to_del_df_2, axis=1, inplace=True)

Посмотрим на типы данных в обоих датасетах

In [34]:
# Функция для поиска различий типов данных
def find_data_type_discrepancy(data_1, data_2):
    """
    Функция ищет различия в типах данных 
    между датасетами и выводит информацию на экран.
    Ничего не возвращает
    :param train: DataFrame object with training data
    :param test: DataFrame object with testing data
    """
    
    data_1_dtypes_df = data_1.dtypes.reset_index()
    data_1_dtypes_df.columns = 'col_name type_data_1'.split(' ')


    data_2_dtypes_df = data_2.dtypes.reset_index()
    data_2_dtypes_df.columns = 'col_name type_data_2'.split(' ')


    merged_df = data_1_dtypes_df.merge(data_2_dtypes_df, how='outer',
                                       left_on='col_name', right_on='col_name')
    merged_df

    print(f"{'Расхождение типов данных:'}")
    print(f"{'Признак'.ljust(21)} | {'data_1'.ljust(15)} | {'data_2'.ljust(15)}")
    print('-'*48)

    for index, row in merged_df.iterrows():
        if str(row['type_data_1']) == 'nan' or str(row['type_data_2']) == 'nan':
            print(f"{str(row['col_name']).ljust(21)} | {str(row['type_data_1']).ljust(15)} | {str(row['type_data_2']).ljust(15)}")

        elif row['type_data_1'] != row['type_data_2']:
            print(f"{str(row['col_name']).ljust(21)} | {str(row['type_data_1']).ljust(15)} | {str(row['type_data_2']).ljust(15)}")


In [35]:
find_data_type_discrepancy(df_1, df_2)

Расхождение типов данных:
Признак               | data_1          | data_2         
------------------------------------------------
section               | nan             | object         
url_saleid            | nan             | object         


Отлично, различий нет, можно конкатенировать датасеты

In [37]:
df = df_1.append(df_2, sort=False).reset_index(drop=True)
df.shape

(57704, 35)

### Запись в файл

In [50]:
def write_to_csv(data):
    date_str = datetime.now().strftime('%d_%m_%Y')
    df.to_csv(f"concatenated_dataset_{date_str}.csv", index=False, encoding='utf-8')

In [ ]:
# Write to csv-file
write_to_csv(df)

На этом конкатенация двух датасетов и исправление данных в признаке ПТС завершена.  